# FIT5148 - Distributed Databases and Big Data
# Assignment 2 Task C2- Solution Workbook

**Instructions:**

    You will be using Python 3.
    You can introduce new cells as necessary.

**Your details**

    Name: Moon Byeong Kim

    Student ID: 26389126

    Name: Hun Tae Lee
    Student ID: 22568735

Let's get started

## Import Libraries

In [1]:
from pymongo import MongoClient
from pprint import pprint
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
climateHeader = ['DataType','Station','   Air Temperature(Celcius)','  Relative Humidity',
                 '  WindSpeed  (knots)',' Max Wind Speed','   MAX  ','  MIN  ','Precipitation']
fireHeader = ['DataType','Latitude','Longitude','Surface Temperature (kelvin)','Power,Confidence',
              'Surface Temperature (Celcius)']
#Required Methods goes here-----

#split the received data and return time and data. Time will be the key.
def splitData(data):
    attributes = data.split(',')
    time = attributes[-1]
    return time, attributes
    
#def rddForEach(time, rdd):
#    rdd.foreach(writeToMongo)
'''    
def writeToMongo(item):
    #findout cdata or fdata
    cdata = item[1][0]
    fdata = item[1][1]
    #create dict for climate
    climate = dict()
    #using header, append data
    for index in range(1,9):
        climate.update({climateHeader[index]:cdata[index]})

    #create array for fires
    fires = []
    #for each fire in fdata
    for f in fdata:
        #create dict
        fire = dict()
        #using header, append data
        for index in range(1,7):
            fire.update({fireHeader[index]:f[index]})
        #append fire data to fire array
        fires.append(fire)
    #append fire array to climate
    climate.update({"fires":fires})
    #write to mongo
    client = MongoClient()
    db = client.taskc
    joinedData = db.joinedData
    joinedData.insert(climate)
'''    


#End of Methods

#host and port - make sure port number is same as send
host = 'localhost'
port = 9999


batch_interval = 5 #interval of 5 seconds
sc = SparkContext('local[2]','test') #two threads
#main entrypoint of sparkstreaming functionality. 
#this provides methods used to create DStreams from various input sources.
ssc = StreamingContext(sc,batch_interval)
#SocketTextStream - Define DStreams received from server through TCP socket.
data = ssc.socketTextStream(host,port)
#split each line into lines, split by \n
lines = data.flatMap(lambda line: line.split('\n'))
#print each batch
lines.pprint()

#filtering the lines by checking whether it starts with cdata or fdata. 
climateSet = lines.filter(lambda head: head.startswith('cdata'))
fireSet = lines.filter(lambda head: head.startswith('fdata'))

#mapping the data
climateSet = climateSet.map(splitData)
fireSet = fireSet.map(splitData)

#merging the fireset which has same key
fireSetMerge = fireSet.groupByKey()

#joinging the firesets to climateset
joinedSet = climateSet.join(fireSetMerge)

#joinedSet.foreachRDD(rddForEach)




#start the computation
ssc.start()
#wait for the computation
ssc.awaitTermination()

-------------------------------------------
Time: 2018-05-21 12:51:15
-------------------------------------------
cdata,948700,19,56.8,7.9,11.1,   72.0*,  61.9*, 0.00I,1526871074
fdata,-37.966,145.051,341.8,26.7,78,68,1526871074
fdata,-35.541,143.311,336.3,62,82,63,1526871074
fdata,-35.554,143.307,326.8,23.8,67,53,1526871074
fdata,-35.543,143.316,340.4,84.2,86,67,1526871074
fdata,-37.708,145.1,327.8,16.2,80,54,1526871074

-------------------------------------------
Time: 2018-05-21 12:51:20
-------------------------------------------
cdata,948700,15,50.7,9.2,13,                                                                                                                                                                                                                                                                                                                                                                                                                                                

KeyboardInterrupt: 